# LastMile Instrumentor for LangChain "Plan-and-Execute" agent


In this notebook, we showcase how to use the **LastMile Tracing SDK** to auto-instrument tracing for a LangChain "Plan-and-Execute" agent. With tracing automatically setup, you can easily debug your application using LastMile's RAG Workbench.

The LangChain "Plan-and-Execute" agent accomplishes an objective by first planning what to do and then executing the sub-tasks. LastMile autoinstrumentation is used to trace and monitor the execution of the agent.

## Notebook Outline
* [Step 1: Setup](#setup)
* [Step 2: Configure the LastMile Instrumentor](#step2)
* [Step 3: Define Tools for LangChain Agent](#step3)
* [Step 4: Run example with LangChain Agent](#step4)
* [Step 5: View Trace Data in RAG Debugger](#step5)

<a name="setup"></a>
# Step 1: Setup

To begin, we need to install a few packages including langchain and lastmile-eval.

In [ ]:
!pip install langchain
!pip install langchain_community
!pip install langchain_core
!pip install langchain_experimental
!pip install langchain_openai
!pip install wikipedia
!pip install numexpr

# LastMile AI Evaluation SDK imports
!pip install lastmile_eval
!pip install "tracing-auto-instrumentation[langchain]" --upgrade

Import the necessary libraries for this example.

In [9]:
from langchain.chains import LLMMathChain
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import Tool
from langchain_experimental.plan_and_execute import (
    PlanAndExecute,
    load_agent_executor,
    load_chat_planner,
)
from langchain_openai import ChatOpenAI, OpenAI

Before we start this tutorial, we need the following tokens/keys:

* LastMile AI API Token: Go to the [LastMile Settings page](https://lastmileai.dev/settings?page=tokens). You will need to first create a LastMile AI account.
* OpenAI API Key: Go to [OpenAI API Keys page](https://platform.openai.com/account/api-keys) to create and access your OpenAI API Key.

You can either set these explicitly (uncomment the lines below), or save them in a `.env` file within this project directory.


In [10]:
import dotenv
dotenv.load_dotenv()

import os

try:
    # Reading secrets from Google Colab
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] =  userdata.get('OPENAI_API_KEY')
    os.environ['LASTMILE_API_TOKEN'] =  userdata.get('LASTMILE_API_TOKEN')
except ModuleNotFoundError:
    # Else reading environment variables
    import dotenv
    dotenv.load_dotenv()

<a name="step2"></a>

# Step 2: Configure the LastMile Instrumentor

Next, we create an instance of `LangChainInstrumentor` with a project name. The `instrument()` method is called to instrument the code for tracing and monitoring.

In [11]:
from tracing_auto_instrumentation.langchain import LangChainInstrumentor

# Create an instance of LangChainInstrumentor and instrument the code
instrumentor = LangChainInstrumentor(project_name="Langchain Plan-and-Execute")
instrumentor.instrument()

<a name="step3"></a>

# Step 3: Define Tools for Langchain Agent

The tools that the agent will use for executing sub-tasks are defined. In this example, we have two tools:
- "Wikipedia" tool: Uses the `WikipediaAPIWrapper` to look up information on Wikipedia.
- "Calculator" tool: Uses the `LLMMathChain` to perform mathematical calculations.

In [12]:
# Define the tools that the agent will use for executing sub-tasks
wikipedia = WikipediaAPIWrapper()
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name="Wikipedia",
        func=wikipedia.run,
        description="useful for when you need to look up information on Wikipedia",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to perform mathematical calculations",
    ),
]


<a name="step4"></a>

# Step 4: Run example with LangChain Agent

Let's try an example with our LangChain agent. We will ask a simple question "What is hte population of the capital city of the country where the Eiffel Tower is located? Calculate the square root of that number."

The agent needs to look up the population (using Wikipedia) and then also make a calculation.

In [13]:
model = ChatOpenAI(temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor)

# ## Run Example

# Run the agent with a sample question
question = "What is the population of the capital city of the country where the Eiffel Tower is located? Calculate the square root of that number."
result = agent.run(question)



> Entering new AgentExecutor chain...
Thought: I can provide the information by looking up the location of the Eiffel Tower on Wikipedia.

Action:
```
{
  "action": "Wikipedia",
  "action_input": "Eiffel Tower location"
}
```
Observation: Page: Eiffel Tower
Summary: The Eiffel Tower ( EYE-fəl; French: Tour Eiffel [tuʁ ɛfɛl] ) is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower from 1887 to 1889.
Locally nicknamed "La dame de fer" (French for "Iron Lady"), it was constructed as the centerpiece of the 1889 World's Fair, and to crown the centennial anniversary of the French Revolution. Although initially criticised by some of France's leading artists and intellectuals for its design, it has since become a global cultural icon of France and one of the most recognisable structures in the world.  The tower received 5,889,000 visitors in 2022. The Eiffel Tower is the most visited mon

In [14]:
print(result)

The square root of the population of Paris is approximately 1450.05


<a name="step5"></a>

# Step 5: View Trace Data in RAG Debugger
Now we can view the trace data of our LangChain agent in a UI!
#### From your terminal:

Export your LASTMILE_API_TOKEN

```bash
export LASTMILE_API_TOKEN="<your-api-token>"
```

Run this CLI command to access the UI

```bash
rag-debug launch
```

Step 1 - In the top-right corner, select your Project
Step 2 - Click the bug icon on the Trace to see the full trace view

<img width="1800" alt="Langchain Plan-and-Execute Traces" src="https://github.com/lastmile-ai/eval-cookbook/assets/129882602/6349c02b-0402-4071-8719-003670eca52a">


Let's click into the Trace.

<img width="1800" alt="Langchain Plan-and-Execute Trace Details" src="https://github.com/lastmile-ai/eval-cookbook/assets/129882602/4b14a989-5b75-4874-ab2b-b6e31f074935">


Here we can see all the spans auto-generated for us. This can help us debug and pinpoint issues in our application, especially if we add additional logging on top of the auto-instrumentor.